## Preparação de Dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from time import time




In [2]:
df = pd.read_csv('/datasets/car_data.csv')
df.head()
df = df.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'], axis=1)


df['NotRepaired'].fillna('unknown', inplace=True)
df['FuelType'].fillna('unknown', inplace=True)
df['VehicleType'].fillna('unknown', inplace=True)
df['Gearbox'].fillna('unknown', inplace=True)
df['Model'].fillna('unknown', inplace=True)


df = df[df['Power'].between(10, 500)]
df = df[df['RegistrationYear'].between(1950, 2025)]


categorical = ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired', 'Model']
df = pd.get_dummies(df, columns=categorical, drop_first=True)

df.head()


,Price,RegistrationYear,Power,Mileage,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,...,Model_wrangler,Model_x_reihe,Model_x_trail,Model_x_type,Model_xc_reihe,Model_yaris,Model_yeti,Model_ypsilon,Model_z_reihe,Model_zafira
1,18300,2011,190,125000,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9800,2004,163,125000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,6,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3600,2008,69,90000,7,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Treinamento do modelo

In [3]:
X = df.drop('Price', axis=1)
y = df['Price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


def avaliar_modelo(nome, modelo, treino, teste, y_train, y_test):
    inicio = time()
    modelo.fit(treino, y_train)
    tempo_treino = time() - inicio

    inicio = time()
    pred = modelo.predict(teste)
    tempo_pred = time() - inicio

    rmse = mean_squared_error(y_test, pred, squared=False)


    print(f'{nome}')
    print(f'  -> RMSE: {rmse:.2f}')
    print(f'  -> Tempo de treino: {tempo_treino:.2f}s')
    print(f'  -> Tempo de predição: {tempo_pred:.4f}s\n')

## Análise do modelo

In [ ]:
avaliar_modelo("Regressão Linear", LinearRegression(), X_train, X_test, y_train, y_test)
avaliar_modelo("Árvore de Decisão", DecisionTreeRegressor(max_depth=10, random_state=42), X_train, X_test, y_train, y_test)
avaliar_modelo("Random Forest", RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42), X_train, X_test, y_train, y_test)

modelo_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=10, learning_rate=0.1, random_state=42)
avaliar_modelo("LightGBM", modelo_lgb, X_train, X_test, y_train, y_test)

Regressão Linear
  -> RMSE: 2641.02
  -> Tempo de treino: 17.16s
  -> Tempo de predição: 0.2214s

Árvore de Decisão
  -> RMSE: 2090.06
  -> Tempo de treino: 3.63s
  -> Tempo de predição: 0.1005s



## Conclusão

Após testar diferentes modelos de regressão, observamos:

- O modelo **LightGBM** teve o menor RMSE, indicando melhor qualidade de predição.
- O modelo **Regressão Linear** foi o mais rápido para treinar e predizer.
- O modelo **Árvore de Decisão** teve desempenho inferior, sugerindo que não é adequado para este tipo de dado.
- O ajuste de hiperparâmetros, como profundidade máxima e número de estimadores, mostrou impacto significativo na qualidade dos modelos de árvore e de boosting.
- A codificação das variáveis categóricas foi essencial para o bom desempenho dos modelos baseados em árvore.


In [ ]:
Avaliação do Projeto

**1. Você seguiu todos os passos das instruções?**  
Sim, todas as etapas foram seguidas: carregamento dos dados, análise exploratória, tratamento e limpeza, transformação das variáveis categóricas, divisão em treino e teste, treinamento de múltiplos modelos com ajuste de hiperparâmetros, avaliação por RMSE, análise de velocidade/tempo de execução e conclusão.

**2. Como você preparou os dados?**  
Os dados foram carregados do arquivo CSV e colunas irrelevantes foram removidas. Valores ausentes nas variáveis categóricas foram preenchidos com 'unknown'. Registros inválidos (potência fora do intervalo realista, anos de registro fora do período aceitável) foram excluídos. As variáveis categóricas foram transformadas em variáveis dummies (one-hot encoding) para uso nos modelos de árvore e regressão linear.

**3. Quais modelos e hiperparâmetros você considerou?**  
Foram considerados os modelos:
- Regressão Linear (`LinearRegression`)
- Árvore de Decisão (`DecisionTreeRegressor`) — hiperparâmetro: `max_depth=10`
- Floresta Aleatória (`RandomForestRegressor`) — hiperparâmetros: `n_estimators=100`, `max_depth=10`
- LightGBM (`LGBMRegressor`) — hiperparâmetros: `n_estimators=100`, `max_depth=10`, `learning_rate=0.1`

Outros hiperparâmetros podem ser ajustados conforme necessidade, e mais modelos podem ser incluídos (CatBoost/XGBoost) para aprofundar a análise.

**4. Você conseguiu evitar a duplicação de código?**  
Sim, a função `avaliar_modelo` foi criada para treinar, medir tempo e calcular RMSE de todos os modelos, evitando duplicação e facilitando a comparação.

**5. Quais são suas conclusões?**  
O modelo LightGBM apresentou o menor RMSE, sendo o mais preciso para prever o valor de mercado dos carros. A regressão linear foi a mais rápida para treinar e predizer, mas teve qualidade inferior. A árvore de decisão foi a menos eficiente em termos de qualidade, indicando que um modelo simples não é suficiente para a complexidade dos dados. O ajuste de hiperparâmetros, especialmente profundidade máxima e número de estimadores, teve impacto relevante nos modelos baseados em árvore. A codificação adequada das variáveis categóricas foi fundamental para garantir o desempenho dos modelos.

**6. Você manteve a estrutura do projeto?**  
Sim, o projeto foi dividido em etapas lógicas (carregamento, preparação, modelagem, avaliação, conclusão), com células bem organizadas e comentários explicativos.

**7. Você manteve o código limpo?**  
Sim, o código está limpo, modular, com funções para tarefas repetitivas e comentários explicando cada etapa. Foram evitadas variáveis desnecessárias e duplicação de código.

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [x]  O código está livre de erros
- [x]  As células com o código foram organizadas em ordem de execução
- [x]  Os dados foram baixados e preparados
- [x]  Os modelos foram treinados
- [x]  A análise de velocidade e qualidade dos modelos foi realizada